In [66]:
import pandas as pd
import os
import random
import numpy as np
import json
import psycopg2
from dotenv import load_dotenv
import transformers
from transformers import CLIPProcessor, CLIPModel
import torch

In [67]:
# loading variables from .env file
load_dotenv("../../private_data/.env") 

# PARENT gets us to the root of the project
PARENT = "./../../"

FOLDER_TABLE = PARENT + os.getenv("FOLDER_TABLE")
FILE_FABRITIUS_DATA = PARENT + os.getenv("FILE_FABRITIUS_DATA")
FILE_FABRITIUS_DATA_FILTERED = PARENT + os.getenv("FILE_FABRITIUS_DATA_FILTERED")
FILE_FABRITIUS_DATA_FILTERED_DOWNLOADED = PARENT + os.getenv("FILE_FABRITIUS_DATA_FILTERED_DOWNLOADED")
FOLDER_FIGURES = PARENT + os.getenv("FOLDER_FIGURES")
IMAGES_FOLDER = PARENT + os.getenv("IMAGES_FOLDER")
RECORD_IDS_TESTING_SET = PARENT + os.getenv("RECORD_IDS_TESTING_SET")
RECORD_IDS_VALIDATION_SET = PARENT + os.getenv("RECORD_IDS_VALIDATION_SET")
WRITTEN_CAPTIONS_TESTING_SET = PARENT + os.getenv("WRITTEN_CAPTIONS_TESTING_SET")
WRITTEN_CAPTIONS_VALIDATION_SET = PARENT + os.getenv("WRITTEN_CAPTIONS_VALIDATION_SET")
FILE_FABRITIUS_ICONOGRAPHIES_JSON = PARENT + os.getenv("FILE_FABRITIUS_ICONOGRAPHIES_JSON")
EMBEDDINGS_FOLDER = PARENT + os.getenv("EMBEDDINGS_FOLDER")
MODELS_FOLDER = PARENT + os.getenv("MODELS_FOLDER")

DB_INPUT_ARTPIECES = PARENT + os.getenv("DB_INPUT_ARTPIECES")
DB_INPUT_ARTISTS = PARENT + os.getenv("DB_INPUT_ARTISTS")
DB_INPUT_SUBJECTMATTER = PARENT + os.getenv("DB_INPUT_SUBJECTMATTER")

FILE_SUBJECTMATTERS_PARSED = PARENT + os.getenv("FILE_SUBJECTMATTERS_PARSED")

PROPER_NOUNS_DATASET_EXPLODED = PARENT + os.getenv("PROPER_NOUNS_DATASET_EXPLODED")
PROPER_NOUNS_DATASET_EXPLODED_UNIQUE_RECORDID = PARENT + os.getenv("PROPER_NOUNS_DATASET_EXPLODED_UNIQUE_RECORDID")
PROPER_NOUNS_DATASET_ATTACHED = PARENT + os.getenv("PROPER_NOUNS_DATASET_ATTACHED")

def get_db_config():
    return {
        "host": os.getenv("DB_HOST"),
        "port": os.getenv("DB_PORT"),
        "name": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
    }

##

def fixPath(path):
    return path.replace(".././", "../")

In [68]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [69]:
model_name_db = "art-mini"
base_name = os.getenv(f"{model_name_db}_model_name")
processor = CLIPProcessor.from_pretrained(base_name)
model = CLIPModel.from_pretrained(base_name).to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained(base_name)
#weights_path = PARENT + os.getenv(f"{model_name_db}_weights_path")
#model.load_state_dict(torch.load(weights_path))

In [70]:
def get_term(db_config, table_name):
    conn = None
    try:
        conn = psycopg2.connect(
            host=db_config["host"],
            port=db_config["port"],
            database=db_config["name"],
            user=db_config["user"],
            password=db_config["password"]
        )
        cur = conn.cursor()
        cur.execute(f"SELECT recordid, values FROM {table_name}_flat WHERE array_length(values, 1) IS NOT NULL;")
        data = cur.fetchall()
        cur.close()
        return data
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [71]:
SUBJECT_TERMS = get_term(get_db_config(), "subjectterms")
SUBJECT_TERMS = pd.DataFrame(SUBJECT_TERMS, columns=["recordid", "values"])
SUBJECT_TERMS = SUBJECT_TERMS.explode("values")
SUBJECT_TERMS

,recordid,values
0,64,scène
0,64,homme
0,64,femme
0,64,enfant
0,64,vêtement
...,...,...
3608,11638,arbre
3609,11646,figure
3609,11646,homme
3609,11646,en buste


In [72]:
CONCEPTUAL_TERMS = get_term(get_db_config(), "conceptualterms")
CONCEPTUAL_TERMS = pd.DataFrame(CONCEPTUAL_TERMS, columns=["recordid", "values"])
CONCEPTUAL_TERMS = CONCEPTUAL_TERMS.explode("values")
CONCEPTUAL_TERMS

,recordid,values
0,79,Robert Schumann
1,80,androgyne
2,82,hybride fabuleux
2,82,sphinx
3,119,martyre
...,...,...
667,11231,mort
667,11231,fuite
667,11231,délivrance
668,11594,donateur


In [73]:
ICONOGRAPHIC_TERMS = get_term(get_db_config(), "iconographicterms")
ICONOGRAPHIC_TERMS = pd.DataFrame(ICONOGRAPHIC_TERMS, columns=["recordid", "values"])
ICONOGRAPHIC_TERMS = ICONOGRAPHIC_TERMS.explode("values")
ICONOGRAPHIC_TERMS

,recordid,values
0,64,scène biblique
0,64,[SO]
0,64,Nouveau Testament
0,64,Evangiles
0,64,Passion
...,...,...
2815,10926,Suisse
2815,10926,pape
2815,10926,Vatican
2816,11805,scène


In [74]:
def is_proper_nouns(term):
    # A term is a proper noun if it starts with a capital letter and is not a single letter
    return len(term) > 1 and term[0].isupper() and term[0]!="["

In [75]:
SUBJECT_TERMS = SUBJECT_TERMS[SUBJECT_TERMS["values"].apply(is_proper_nouns)]
SUBJECT_TERMS

,recordid,values
174,564,Madelon
376,1162,David Lloyd George
376,1162,Verenigd Koninkrijk
407,1218,Ordre de Malte
430,1251,Chine
494,1387,Antiquité
501,1405,Corne d'abondance
873,6187,Paysage
1517,4532,Toison d'Or
1517,4532,Jarretière


In [76]:
CONCEPTUAL_TERMS = CONCEPTUAL_TERMS[CONCEPTUAL_TERMS["values"].apply(is_proper_nouns)]
CONCEPTUAL_TERMS

,recordid,values
0,79,Robert Schumann
3,119,Marie-Anne Charlotte Corday
3,119,Louis XVI
5,126,Rome
5,126,Marcellus
...,...,...
654,10297,Orientalisme
656,10487,Espagne
656,10487,Séville
661,10800,Waterloo


In [77]:
ICONOGRAPHIC_TERMS = ICONOGRAPHIC_TERMS[ICONOGRAPHIC_TERMS["values"].apply(is_proper_nouns)]
ICONOGRAPHIC_TERMS

,recordid,values
0,64,Nouveau Testament
0,64,Evangiles
0,64,Passion
0,64,Calvaire
0,64,Crucifixion
...,...,...
2813,10918,Mer du Nord
2814,10923,François I
2814,10923,Allemagne
2815,10926,Suisse


In [88]:
SUBJECT_TERMS_cp = pd.DataFrame(columns=["term", "origin"])
SUBJECT_TERMS_cp["term"] = list(set(SUBJECT_TERMS["values"].tolist()))
SUBJECT_TERMS_cp["origin"] = "subjectterms"

CONCEPTUAL_TERMS_cp = pd.DataFrame(columns=["term", "origin"])
CONCEPTUAL_TERMS_cp["term"] = list(set(CONCEPTUAL_TERMS["values"].tolist()))
CONCEPTUAL_TERMS_cp["origin"] = "conceptualterms"

ICONOGRAPHIC_TERMS_cp = pd.DataFrame(columns=["term", "origin"])
ICONOGRAPHIC_TERMS_cp["term"] = list(set(ICONOGRAPHIC_TERMS["values"].tolist()))
ICONOGRAPHIC_TERMS_cp["origin"] = "iconographicterms"

MERGED_cp = pd.concat([SUBJECT_TERMS_cp, CONCEPTUAL_TERMS_cp, ICONOGRAPHIC_TERMS_cp], ignore_index=True)
# Group by values
MERGED_cp = MERGED_cp.groupby("term").agg({"origin": lambda x: list(set(x))}).reset_index()
multi_origins_terms = MERGED_cp[MERGED_cp["origin"].apply(lambda x: len(x) > 1)]["term"].tolist()

# Remove the terms that are in multi_origins_terms
SUBJECT_TERMS_cp = SUBJECT_TERMS_cp[~SUBJECT_TERMS_cp["term"].isin(multi_origins_terms)]
CONCEPTUAL_TERMS_cp = CONCEPTUAL_TERMS_cp[~CONCEPTUAL_TERMS_cp["term"].isin(multi_origins_terms)]
ICONOGRAPHIC_TERMS_cp = ICONOGRAPHIC_TERMS_cp[~ICONOGRAPHIC_TERMS_cp["term"].isin(multi_origins_terms)]

number_of_terms_subjectterms = len(SUBJECT_TERMS_cp["term"].tolist())
number_of_terms_conceptualterms = len(CONCEPTUAL_TERMS_cp["term"].tolist())
number_of_terms_iconographicterms = len(ICONOGRAPHIC_TERMS_cp["term"].tolist())
number_of_term_common = len(multi_origins_terms)
number_of_terms_total = number_of_terms_subjectterms + number_of_terms_conceptualterms + number_of_terms_iconographicterms + number_of_term_common
print(f"Number of terms in subjectterms: {number_of_terms_subjectterms}")
print(f"Number of terms in conceptualterms: {number_of_terms_conceptualterms}")
print(f"Number of terms in iconographicterms: {number_of_terms_iconographicterms}")
print(f"Number of terms in common: {number_of_term_common}")

Number of terms in subjectterms: 10
Number of terms in conceptualterms: 142
Number of terms in iconographicterms: 1368
Number of terms in common: 145


In [93]:
count_per_subjectmatter = pd.DataFrame(columns=["Field", "Unique proper nouns", "Percentage"])
count_per_subjectmatter.loc[len(count_per_subjectmatter)] = ["Subject Terms", number_of_terms_subjectterms, number_of_terms_subjectterms/number_of_terms_total]
count_per_subjectmatter.loc[len(count_per_subjectmatter)] = ["Conceptual Terms", number_of_terms_conceptualterms, number_of_terms_conceptualterms/number_of_terms_total]
count_per_subjectmatter.loc[len(count_per_subjectmatter)] = ["Iconographic Terms", number_of_terms_iconographicterms, number_of_terms_iconographicterms/number_of_terms_total]
count_per_subjectmatter.loc[len(count_per_subjectmatter)] = ["Appearing in more than 1 field", len(multi_origins_terms), len(multi_origins_terms)/number_of_terms_total]
count_per_subjectmatter.loc[len(count_per_subjectmatter)] = ["Total", number_of_terms_total, 1]
count_per_subjectmatter["Percentage"] = count_per_subjectmatter["Percentage"].apply(lambda x: "{:.2f}\%".format(x*100))
count_per_subjectmatter.to_csv(FOLDER_TABLE + "count_per_subjectmatter.csv", index=False)
count_per_subjectmatter

,Field,Unique proper nouns,Percentage
0,Subject Terms,10,0.60\%
1,Conceptual Terms,142,8.53\%
2,Iconographic Terms,1368,82.16\%
3,Appearing in more than 1 field,145,8.71\%
4,Total,1665,100.00\%


I decided to make two datasets that have the purpose of testing the proper nouns:
1) The first is the "exploded" proper nouns, for example if a recordID has 3 proper nous attached: Jesus, Nel Wouters and Amsterdam, then the dataset would have three entry for that recordID, one for each proper noun. This dataset is closer to a real user query. You don't query for exactly all the people/places you want to see, you query for a single or maybe two terms
2) The second is the "attached" proper noins, for example if a recordID has 3 proper nous attached: Jesus, Nel Wouters and Amsterdam, then the dataset would have one entry for that recordID equal to: "Jesus, Nel Wouters, Amsterdam" (I will maybe remove the ',' if the number of token < 75)

In [91]:
def get_number_of_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [92]:
MERGED_TERMS = pd.concat([SUBJECT_TERMS, CONCEPTUAL_TERMS, ICONOGRAPHIC_TERMS], ignore_index=True)
# Remove duplicates
MERGED_TERMS = MERGED_TERMS.drop_duplicates(subset=["recordid", "values"])
MERGED_TERMS

,recordid,values
0,564,Madelon
1,1162,David Lloyd George
2,1162,Verenigd Koninkrijk
3,1218,Ordre de Malte
4,1251,Chine
...,...,...
4758,10918,Mer du Nord
4759,10923,François I
4760,10923,Allemagne
4761,10926,Suisse


In [28]:
exploded_dataset = pd.DataFrame(columns=["recordIDs", "term", "length_tokenization"])
MERGED_TERMS_grouped_by_term = MERGED_TERMS.groupby("values").agg(list).reset_index()
for index, row in MERGED_TERMS_grouped_by_term.iterrows():
    recordIDs = row["recordid"]
    term = row["values"]
    length_tokenization = get_number_of_tokens(term)
    exploded_dataset.loc[len(exploded_dataset)] = [recordIDs, term, length_tokenization]
exploded_dataset

,recordIDs,term,length_tokenization
0,[8437],Aar,1
1,[1369],Ab Urbe Condita,5
2,[5299],Abigaïl,4
3,"[6622, 7295]",Abraham,1
4,[5354],Abraham Bloemaert,4
...,...,...,...
1660,"[7587, 8256, 625, 1598]",Zélande,3
1661,[1062],Égypte,3
1662,[10189],Élysée Reclus,7
1663,[10241],Énée,3


In [29]:
record_id_to_equals = {}
for index, row in exploded_dataset.iterrows():
    recordIDs = row["recordIDs"]
    for unique_recordID in recordIDs:
        if unique_recordID not in record_id_to_equals:
            record_id_to_equals[unique_recordID] = set()
        record_id_to_equals[unique_recordID].update(recordIDs)

record_id_to_equals

{8437: {8437},
 1369: {126,
  161,
  204,
  385,
  509,
  882,
  1219,
  1369,
  1387,
  1528,
  1551,
  1689,
  2292,
  2660,
  2661,
  2997,
  3470,
  3473,
  4408,
  4409,
  4491,
  4492,
  4494,
  4849,
  5028,
  5266,
  5275,
  5454,
  5498,
  5738,
  5888,
  5904,
  6131,
  6188,
  6243,
  6244,
  6245,
  6247,
  6248,
  6249,
  6269,
  7908,
  7910,
  7913,
  7922,
  7947,
  8051,
  8081,
  8485,
  8499,
  10632,
  10845},
 5299: {243,
  280,
  285,
  382,
  430,
  1246,
  1296,
  1688,
  1855,
  1985,
  2432,
  2515,
  2575,
  2836,
  4079,
  4663,
  4988,
  5036,
  5299,
  5402,
  5407,
  5426,
  5495,
  5652,
  5673,
  6387,
  6407,
  6487,
  6511,
  6621,
  6622,
  6789,
  7259,
  7295,
  7658,
  7923,
  8112,
  8259,
  8390,
  10297,
  10330,
  10841,
  10870,
  10874,
  11231},
 6622: {243,
  280,
  285,
  382,
  430,
  1246,
  1296,
  1688,
  1855,
  1985,
  2432,
  2515,
  2575,
  2836,
  4079,
  4663,
  4988,
  5036,
  5299,
  5402,
  5407,
  5426,
  5495,
  5652,
  567

In [30]:
"""
exploded_dataset has a columns called recordIDs with a list of recordIDs.
But when we are going to benchmark a model we also need to have a unique recordID for each term.
Each row with multiple recordIDs should be exploded into m rows with a single recordID.
"""

exploded_dataset_with_unique_recordIDs = pd.DataFrame(columns=["recordID", "term", "length_tokenization"])
for index, row in exploded_dataset.iterrows():
    recordIDs = row["recordIDs"]
    term = row["term"]
    length_tokenization = row["length_tokenization"]
    for recordID in recordIDs:
        exploded_dataset_with_unique_recordIDs.loc[len(exploded_dataset_with_unique_recordIDs)] = [recordID, term, length_tokenization]

exploded_dataset_with_unique_recordIDs

,recordID,term,length_tokenization
0,8437,Aar,1
1,1369,Ab Urbe Condita,5
2,5299,Abigaïl,4
3,6622,Abraham,1
4,7295,Abraham,1
...,...,...,...
4747,10189,Élysée Reclus,7
4748,10241,Énée,3
4749,2897,États Unis d'Amérique,9
4750,7429,États Unis d'Amérique,9


In [31]:
# Count of the length_tokenization
under_75_tokens = exploded_dataset[exploded_dataset["length_tokenization"] < 75].shape[0]
over_75_tokens = exploded_dataset[exploded_dataset["length_tokenization"] >= 75].shape[0]
print(f"Number of terms with less than 75 tokens: {under_75_tokens}")
print(f"Number of terms with 75 or more tokens: {over_75_tokens}")

Number of terms with less than 75 tokens: 1665
Number of terms with 75 or more tokens: 0


In [32]:
MERGED_TERMS_ATTACHED = MERGED_TERMS.groupby("recordid").agg(list).reset_index()
MERGED_TERMS_ATTACHED
attached_dataset = pd.DataFrame(columns=["recordID", "term", "length_tokenization"])
for index, row in MERGED_TERMS_ATTACHED.iterrows():
    recordIDs = row["recordid"]
    term = ", ".join(row["values"])
    length_tokenization = get_number_of_tokens(term)
    attached_dataset.loc[len(attached_dataset)] = [recordIDs, term, length_tokenization]
attached_dataset

,recordID,term,length_tokenization
0,64,"Nouveau Testament, Evangiles, Passion, Calvair...",29
1,78,Louise van der Hecht,5
2,79,Robert Schumann,3
3,81,Marguerite Khnopff,6
4,105,"Ariane, Bacchus, Cupidon",9
...,...,...,...
1900,11252,Namur,2
1901,11521,"Persée, Andromède, Céto",10
1902,11525,"Marie-Anne Hortemels, Nicolas-Henri Tardieu",13
1903,11533,"Nel Wouters, Rik Wouters, Amsterdam",11


In [33]:
# Count of the length_tokenization
under_75_tokens = attached_dataset[attached_dataset["length_tokenization"] < 75].shape[0]
over_75_tokens = attached_dataset[attached_dataset["length_tokenization"] >= 75].shape[0]
print(f"Number of terms with less than 75 tokens: {under_75_tokens}")
print(f"Number of terms with 75 or more tokens: {over_75_tokens}")

Number of terms with less than 75 tokens: 1901
Number of terms with 75 or more tokens: 4


In [34]:
# Export both datasets to CSV
exploded_dataset.to_csv(PROPER_NOUNS_DATASET_EXPLODED, index=False)
exploded_dataset_with_unique_recordIDs.to_csv(PROPER_NOUNS_DATASET_EXPLODED_UNIQUE_RECORDID, index=False)
attached_dataset.to_csv(PROPER_NOUNS_DATASET_ATTACHED, index=False)